# Web scraping stock market news for Sentiment Analysis

## 1. Introduction

Stock market news articles from 2014-2021 will be collected by dynamic web scraping from [Investing.com](https://uk.investing.com/equities/astrazeneca-news) using a combination of Selenium library to automate browser interaction enabling data extraction by Beautiful Soup.




## 2. Install/import libraries

In [ ]:
!pip install htmldate
!pip install twython
!pip3 install newspaper3k
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 853.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=9f25f1784e1757cee48fe28c009b2c4935c5aeb86a3b306f2fd72fa9251ac3aa
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size

In [ ]:
import pandas as pd
import numpy as np
import time
import twython
import requests
import nltk
import warnings
warnings.filterwarnings('ignore')

from htmldate import find_date
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
from newspaper import Article
from selenium.webdriver.common.by import By

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.lWqiuFbTWG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.qDUI8Wbzyg/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.BXNtb6Ubqi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


## 3. Data collection



In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=chrome_options)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_newslinks(company, page_number):

    url = f"https://uk.investing.com/equities/{company}-news/{page_number}"
    wd.get(url)

    href = []

    # scroll all the way to the bottom

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = wd.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                " window.pageYOffset : (document.documentElement ||"
                " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        wd.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = wd.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                " window.pageYOffset : (document.documentElement ||"
                " document.body.parentNode || document.body);"))

    cleaned_links = []

    # Iterate through all the articles on the page
    for article_number in range(1,10):
      article_xpath = f'/html/body/div[5]/section/div[8]/article[{article_number}]'
      try:
        article = WebDriverWait(wd, 60).until(
          EC.presence_of_element_located((By.XPATH, article_xpath))
        )
        article_html = article.get_attribute('innerHTML')
        soup = BeautifulSoup(article_html, "lxml")
        for link in soup.find_all('a'):
            # Get the href
            partial_link = link.get('href')
            if 'https' in partial_link:
                cleaned_links.append(partial_link)
            # Some links are 'internal' to the page and don't have https in them. The web page must be appended to these links
            elif partial_link[0] == '/':
                cleaned_links.append('https://uk.investing.com/'+partial_link)
      except NoSuchElementException:
            print(f"Element not found for article {article_number}. Skipping...")


    return np.unique(cleaned_links)



In [ ]:
# Create empty list to append URLs    return np.unique(cleaned_links)
all_company_urls = []
for page in range(1, 99):
    results = get_newslinks('astrazeneca', page)
    all_company_urls.extend(results)
all_company_urls

TimeoutException: ignored

In [ ]:
all_company_urls = [
 'https://invst.ly/tb-6x',
 'https://invst.ly/tb-n4',

 'https://invst.ly/tbxxc',
 'https://invst.ly/tbyb8',



 'https://invst.ly/tcjd5',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-jan-4-2274638',


 'https://invst.ly/tavzk',
 'https://invst.ly/tazh-',
 'https://invst.ly/tb0n-',

 'https://invst.ly/tb0pu',
 'https://invst.ly/tbgq4',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-dec-31-2273851',
 'https://invst.ly/tao66',
 'https://invst.ly/tapfh',
 'https://invst.ly/tapq5',

 'https://invst.ly/tar6p',
 'https://invst.ly/tara2',
 'https://invst.ly/targ5',
 'https://invst.ly/tatz3',

 'https://uk.investing.com//news/economy/asian-shares-pause-recent-rally-euro-near-212year-high-2273311',

 'https://invst.ly/tamuz',
 'https://invst.ly/tan9u',

 'https://invst.ly/tanns',
 'https://invst.ly/tanp5',
 'https://invst.ly/tanv0',
 'https://invst.ly/tao2d',

 'https://invst.ly/tao66',

 'https://invst.ly/tajpx',
 'https://invst.ly/tajxo',

 'https://invst.ly/takl3',

 'https://invst.ly/tam45',
 'https://invst.ly/tam6k',
 'https://invst.ly/tamsg',
 'https://uk.investing.com//news/economy/mcconnell-defies-trump-uk-oks-astra-vaccine-bitcoin-ath--whats-up-in-markets-2273468',
 'https://invst.ly/ta6vz',

 'https://invst.ly/ta8yt',
 'https://invst.ly/ta92r',
 'https://invst.ly/taahy',

 'https://invst.ly/tac77',

 'https://invst.ly/tady6',
 'https://invst.ly/taj2p',
 'https://invst.ly/t9-iv',
 'https://invst.ly/t9rte',

 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-050-2271496',

 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-dec-22-2270544',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-dec-21-2269910',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-dec-22-2270545',

 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-dec-21-2269886',

 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-068-2266577',

 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-dec-10-2264610',

 'https://uk.investing.com//news/stock-market-news/stockbeat-just-when-you-thought-it-was-safe-to-get-back-in-the-water-2264034',
 'https://uk.investing.com//news/stock-market-news/stockbeat-just-when-you-thought-it-was-safe-to-get-back-in-the-water-2264034#comments',

 'https://www.proactiveinvestors.co.uk/companies/news/935808/ftse-100-closes-a-hair-higher-as-investors-wait-to-see-outcome-of-brexit-talks-935808.html',
 'https://www.proactiveinvestors.co.uk/companies/news/935808/ftse-100-nudges-higher-as-investors-wait-to-see-outcome-of-brexit-talks-935808.html',

 'https://www.proactiveinvestors.co.uk/companies/news/935808/ftse-100-sees-gains-evaporate-as-sterling-recovers-some-poise-935808.html',

 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-036-2261244',

 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-dec-3-2260831',

 'https://uk.investing.com//news/economy/black-friday-arrives-spanish-banks-turn-down-merger-opec-looms--whats-up-in-ma-2258037',
 'https://uk.investing.com//news/economy/europe-confidence-slips-bitcoin-corrects-slack-talk--whats-up-in-markets-2257576',
 'https://uk.investing.com//news/stock-market-news/dow-slips-on-concerns-over-recovery-with-weaker-jobs-restrictions-2254009',
 'https://uk.investing.com//news/stock-market-news/pointcounterpoint-the-case-for-astrazeneca-2258221',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-013-2255629',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-dec-1-2259372',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-nov-24-2255933',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-nov-27-2257959',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-nov-30-2258696',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-nov-30-2258696#comments',
 'https://uk.investing.com//news/coronavirus/astrazeneca-covid19-shot-candidate-shows-promise-among-elderly-in-trials-2253491',
 'https://uk.investing.com//news/coronavirus/indias-serum-says-produced-40-million-doses-of-astrazeneca-covid19-vaccine-2249739',
 'https://uk.investing.com//news/coronavirus/oxfords-covid19-vaccine-trial-results-definitely-before-christmas--investigator-2253539',
 'https://uk.investing.com//news/coronavirus/why-i-volunteered-for-a-covid19-vaccine-trial-2249799',
 'https://uk.investing.com//news/stock-market-news/eu-seals-deal-with-pfizerbiontech-for-supply-of-300-million-doses-of-covid-vaccine-2249271',
 'https://uk.investing.com//news/stock-market-news/eu-to-okay-deal-with-curevac-for-supply-of-up-to-405-million-covid-vaccines-2251697',
 'https://uk.investing.com//news/stock-market-news/exclusive-europe-to-pay-less-than-us-for-pfizer-vaccine-under-initial-deal--source-2249380',
 'https://uk.investing.com//news/stock-market-news/pfizer-shares-drop-as-moderna-vaccine-seen-easier-to-distribute-2251752',
 'https://uk.investing.com//news/stock-market-news/pfizerbiontech-vaccine-deliveries-could-start-before-christmas-2253278',
 'https://uk.investing.com//news/world-news/where-theres-a-will-theres-a-way-as-english-doctors-prepare-covid-vaccine-rollout-2253132',
 'https://uk.investing.com//news/coronavirus/argentina-agrees-deal-for-22-million-doses-of-astrazenecaoxford-covid19-vaccine-2247244',
 'https://uk.investing.com//news/coronavirus/covid19-war-games-the-computer-program-that-could-help-save-your-job-2249277',
 'https://uk.investing.com//news/coronavirus/oxford-ups-covid19-testing-capacity-with-thermo-fisher-deal-2248519',
 'https://uk.investing.com//news/coronavirus/pfizer-and-biontech-say-their-covid19-vaccine-is-over-90-effective-2247711',
 'https://uk.investing.com//news/coronavirus/pfizer-and-biontech-say-their-covid19-vaccine-is-over-90-effective-2247711#comments',
 'https://uk.investing.com//news/economy/investors-bet-vaccine-sparks-revival-in-beaten-down-stocks-2247477',
 'https://uk.investing.com//news/economy/investors-bet-vaccine-sparks-revival-in-beaten-down-stocks-2247477#comments',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-beats-thirdquarter-sales-estimate-backs-2020-outlook-2245734',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-beats-thirdquarter-sales-estimate-backs-2020-outlook-2245734#comments',
 'https://uk.investing.com//news/stock-market-news/healthcare-gold-miners-fed-statement-3-things-to-watch-2245577',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-nov-5-2245740',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-nov-6-2246593',
 'https://uk.investing.com//news/uk-stocksfactors-to-watch-on-nov-10-2248288',
 'https://uk.investing.com//news/coronavirus/britain-expects-verdict-on-pfizers-covid19-vaccine-before-astrazenecas--times-2241595',
 'https://uk.investing.com//news/coronavirus/oxford-covid19-vaccine-trial-results-due-this-year-trial-chief-says-2245168',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sells-right-to-heart-failure-blood-pressure-medicines-for-400-million-2242626',
 'https://uk.investing.com//news/stock-market-news/energy-stocks-drag-ftse-100-lower-as-virus-fears-weigh-2239406',
 'https://uk.investing.com//news/stock-market-news/europe-prepares-for-more-pain-ahead-as-covid-surges-2239463',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-139-2245407',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-098-2239771',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-nov-2-2243648',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-30-2242627',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-oct-29-2241745',
 'https://uk.investing.com//news/coronavirus/astrazeneca-oxford-allowed-to-resume-covid19-vaccine-trial-in-us--wsj-2238879',
 'https://uk.investing.com//news/coronavirus/astrazeneca-oxford-allowed-to-resume-covid19-vaccine-trial-in-us--wsj-2238879#comments',
 'https://uk.investing.com//news/coronavirus/britain-partners-with-oxford-firm-to-assess-coronavirus-vaccine-t-cell-responses-2237611',
 'https://uk.investing.com//news/politics/fact-check-trump-and-bidens-second-and-final-presidential-debate-2238397',
 'https://uk.investing.com//news/politics/fact-check-trump-and-bidens-second-and-final-presidential-debate-2238397#comments',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-its-oxford-vaccine-deal-allows-it-to-add-up-to-20-of-manufacturing-costs-2238960',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-its-oxford-vaccine-deal-allows-it-to-add-up-to-20-of-manufacturing-costs-2238960#comments',
 'https://uk.investing.com//news/stock-market-news/dow-snaps-3week-win-streak-as-stimulus-jitters-stifle-bulls-2238908',
 'https://uk.investing.com//news/stock-market-news/ftse-100-slips-as-british-airways-virus-concerns-weigh-2237693',
 'https://uk.investing.com//news/stock-market-news/ftse-100-slips-as-british-airways-virus-concerns-weigh-2237693#comments',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-21-2236855',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-23-2238429',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-26-2239350',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-oct-22-2237650',
 'https://uk.investing.com//news/coronavirus/britain-moves-closer-to-covid19-vaccine-trials-that-infect-volunteers-2234032',
 'https://uk.investing.com//news/coronavirus/exclusive-astrazeneca-us-covid19-vaccine-trial-may-resume-as-soon-as-this-week--sources-2236632',
 'https://uk.investing.com//news/coronavirus/us-signs-agreement-with-astrazeneca-to-develop-covid19-antibody-treatment-2230163',
 'https://uk.investing.com//news/coronavirus/us-signs-agreement-with-astrazeneca-to-develop-covid19-antibody-treatment-2230163#comments',
 'https://uk.investing.com//news/stock-market-news/as-globe-gallops-into-vaccine-trials-insurers-remain-unfazed-2230732',
 'https://uk.investing.com//news/stock-market-news/interested-in-the-novacyt-share-price-read-this-first-2232298',
 'https://uk.investing.com//news/stock-market-news/london-stocks-track-asian-gains-on-vaccine-optimism-2235217',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-062-2232822',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-12-2230726',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-8-2228669',
 'https://uk.investing.com//news/world-news/panicked-europe-battens-down-against-covid-second-wave-2232407',
 'https://uk.investing.com//news/coronavirus/astrazeneca-could-profit-from-covid19-vaccine-as-early-as-july--ft-2228380',
 'https://uk.investing.com//news/coronavirus/eu-launches-rolling-review-of-pfizerbiontech-covid19-vaccine-2227039',
 'https://uk.investing.com//news/coronavirus/eu-regulator-launches-realtime-review-of-astrazenecas-covid19-vaccine-2224443',
 'https://uk.investing.com//news/coronavirus/exclusive-fda-widens-us-safety-inquiry-into-astrazeneca-coronavirus-vaccine--sources-2224014',
 'https://uk.investing.com//news/coronavirus/exclusive-world-bank-seeks-board-approval-for-12-billion-coronavirus-vaccine-financing-plan-2222821',
 'https://uk.investing.com//news/coronavirus/exclusive-world-bank-seeks-board-approval-for-12-billion-coronavirus-vaccine-financing-plan-2222821#comments',
 'https://uk.investing.com//news/coronavirus/nvidia-building-uk-supercomputer-to-boost-covid19-research-2226412',
 'https://uk.investing.com//news/coronavirus/uk-technical-failure-in-covid19-testing-data-system-now-fixed-minister-says-2226111',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-resumes-vaccine-trial-in-japan-in-talks-with-us-2225065',
 'https://uk.investing.com//news/stock-market-news/canada-has-signed-deal-for-astrazeneca-vaccine-candidate--pm-2220425',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-oct-1-2224068',
 'https://uk.investing.com//news/coronavirus/astrazeneca-says-covid19-vaccine-trial-in-us-still-on-hold-2215359',
 'https://uk.investing.com//news/coronavirus/covid19-vaccine-makers-see-eu-shield-against-sideeffect-claims-2217077',
 'https://uk.investing.com//news/coronavirus/covid19-vaccine-verdicts-loom-as-next-big-market-risk-2216691',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-still-waiting-for-fda-decision-to-resume-us-trial-2218893',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-still-waiting-for-fda-decision-to-resume-us-trial-2218893#comments',
 'https://uk.investing.com//news/stock-market-news/astrazenecamercks-lynparza-wins-eu-panel-recommendation-for-cancer-treatments-2215648',
 'https://uk.investing.com//news/stock-market-news/exclusive-astrazeneca-gets-partial-immunity-in-lowcost-eu-vaccine-deal-2219844',
 'https://uk.investing.com//news/stock-market-news/exclusive-astrazeneca-gets-partial-immunity-in-lowcost-eu-vaccine-deal-2219844#comments',
 'https://uk.investing.com//news/stock-market-news/ftse-100-tracks-global-markets-higher-but-set-for-weekly-losses-2219903',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-sept-24-2218827',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-sept-25-2219842',
 'https://uk.investing.com//news/uk-stocksfactors-to-watch-on-sept-21-2215608',
 'https://uk.investing.com//news/coronavirus/astrazenecas-trial-illnesses-may-not-be-due-to-covid19-shot--oxford-university-2212984',
 'https://uk.investing.com//news/coronavirus/exclusive-astrazeneca-covid19-vaccine-trial-in-us-on-hold-until-at-least-midweek--sources-2211071',
 'https://uk.investing.com//news/coronavirus/germany-grants-biontech-curevac-745-million-to-speed-up-covid19-vaccine-work-2211746',
 'https://uk.investing.com//news/coronavirus/in-coronavirus-vaccine-race-china-inoculates-thousands-before-trials-are-completed-2212306',
 'https://uk.investing.com//news/economy/asian-shares-set-to-dip-as-investors-await-key-data-2211133',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-tagrisso-shown-to-slow-lung-cancer-spreading-to-brain-2215283',
 'https://uk.investing.com//news/stock-market-news/dow-rallies-on-positive-vaccine-news-tech-jumps-2210988',
 'https://uk.investing.com//news/stock-market-news/london-stocks-gain-as-astrazeneca-resumes-vaccine-trials-2210187',
 'https://uk.investing.com//news/stock-market-news/london-stocks-gain-as-astrazeneca-resumes-vaccine-trials-2210187#comments',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-15-2211214',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-16-2212307',
 'https://uk.investing.com//news/coronavirus/astrazeneca-shares-gain-as-coronavirus-vaccine-trials-resume-2210178',
 'https://uk.investing.com//news/coronavirus/astrazenecas-uk-covid19-vaccine-trials-restarted-eu-medical-register-shows-2210360',
 'https://uk.investing.com//news/economic-indicators/us-weekly-jobless-claims-stuck-at-higher-levels-2208312',
 'https://uk.investing.com//news/economy/asian-markets-set-to-come-under-pressure-after-another-down-day-on-wall-street-2206828',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-should-know-if-vaccine-works-by-yearend-if-trials-resume-2208064',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-trials-of-covid-vaccine-resuming-2209792',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-trials-of-covid-vaccine-resuming-2209792#comments',
 'https://uk.investing.com//news/stock-market-news/regenerons-antibody-drug-added-to-uk-recovery-trial-of-covid-treatments-2210439',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-11-2208951',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-14-2210151',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-sept-10-2207969',
 'https://uk.investing.com//news/coronavirus/australia-expects-to-receive-astrazenecas-covid19-vaccine-within-months-2205449',
 'https://uk.investing.com//news/economy/pause-in-astrazeneca-vaccine-trial-not-necessarily-a-setback-says-health-minister-2206915',
 'https://uk.investing.com//news/economy/pause-in-astrazeneca-vaccine-trial-not-necessarily-a-setback-says-health-minister-2206915#comments',
 'https://uk.investing.com//news/economy/top-5-things-to-know-in-the-market-on-wednesday-september-10th-2207109',
 'https://uk.investing.com//news/economy/top-5-things-to-know-in-the-market-on-wednesday-september-10th-2207109#comments',
 'https://uk.investing.com//news/economy/uk-science-adviser-other-vaccine-trials-also-likely-to-be-paused-2207529',
 'https://uk.investing.com//news/economy/uk-science-adviser-other-vaccine-trials-also-likely-to-be-paused-2207529#comments',
 'https://uk.investing.com//news/stock-market-news/european-shares-inch-higher-astrazeneca-tumbles-on-halting-vaccine-trials-2206955',
 'https://uk.investing.com//news/stock-market-news/european-stocks-seen-lower-us-tech-rout-weighs-2206920',
 'https://uk.investing.com//news/stock-market-news/ftse-100-gains-as-weaker-pound-offsets-astrazeneca-drag-2206959',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-8-2205865',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-9-2206902',
 'https://uk.investing.com//news/stock-market-news/us-european-covid-vaccine-developers-make-pledge-to-uphold-testing-rigour-2206134',
 'https://uk.investing.com//news/coronavirus/astrazeneca-signs-deal-with-albany-molecular-to-produce-covid19-vaccine-candidate-2203732',
 'https://uk.investing.com//news/coronavirus/astrazeneca-to-begin-phase-12-clinical-trials-of-coronavirus-vaccine-in-japan-2204328',
 'https://uk.investing.com//news/coronavirus/australias-csl-agrees-to-manufacture-two-covid19-vaccine-candidates-2205341',
 'https://uk.investing.com//news/coronavirus/covid19-vaccine-developers-prepare-joint-safety-pledge--wsj-2204986',
 'https://uk.investing.com//news/coronavirus/factbox-astrazenecas-potential-coronavirus-vaccine-2201687',
 'https://uk.investing.com//news/coronavirus/factbox-astrazenecas-potential-coronavirus-vaccine-2201687#comments',
 'https://uk.investing.com//news/coronavirus/sanofi-france-chief-future-covid19-vaccine-seen-below-10-euros-2205030',
 'https://uk.investing.com//news/coronavirus/sanofi-france-chief-sees-coronavirus-vaccine-priced-below-10-euros-2205219',
 'https://uk.investing.com//news/economy/thanks-for-the-covid-vaccine-but-can-you-say-that-in-estonian-2201597',
 'https://uk.investing.com//news/stock-market-news/stronger-pound-hits-ftse-100-after-best-august-since-2014-2201383',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-sept-1-2201323',
 'https://uk.investing.com//news/coronavirus/astrazeneca-starts-trial-of-covid19-antibody-treatment-2196459',
 'https://uk.investing.com//news/coronavirus/astrazeneca-starts-trial-of-covid19-antibody-treatment-2196459#comments',
 'https://uk.investing.com//news/coronavirus/astrazenecas-covid19-vaccine-candidate-enters-latestage-study-in-us-2201207',
 'https://uk.investing.com//news/coronavirus/australia-locks-in-coronavirus-vaccine-deal-as-new-cases-ease-2192488',
 'https://uk.investing.com//news/coronavirus/eu-pays-336-million-euros-for-doses-of-astrazenecas-potential-covid19-vaccine-2198868',
 'https://uk.investing.com//news/coronavirus/eu-signs-contract-with-astrazeneca-on-supply-of-potential-covid19-vaccine-2198554',
 'https://uk.investing.com//news/coronavirus/eu-signs-contract-with-astrazeneca-on-supply-of-potential-covid19-vaccine-2198554#comments',
 'https://uk.investing.com//news/coronavirus/exclusive-top-fda-official-says-would-resign-if-agency-rubberstamps-an-unproven-covid19-vaccine-2194375',
 'https://uk.investing.com//news/coronavirus/ftse-100-tracks-asia-gains-on-virus-treatment-hopes-astrazeneca-jumps-2195593',
 'https://uk.investing.com//news/coronavirus/ftse-100-tracks-asia-gains-on-virus-treatment-hopes-astrazeneca-jumps-2195593#comments',
 'https://uk.investing.com//news/stock-market-news/ftse-100-gains-on-retail-strength-stronger-sterling-weighs-on-exporters-2194474',
 'https://uk.investing.com//news/stock-market-news/ftse-100-gains-on-retail-strength-stronger-sterling-weighs-on-exporters-2194474#comments',
 'https://uk.investing.com//news/stock-market-news/ftse-100-gains-on-uschina-trade-optimism-2196488',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-aug-25-2196443',
 'https://uk.investing.com//news/coronavirus/astrazeneca-set-to-start-making-400-million-covid19-vaccines-for-latam-early-in-2021-2188899',
 'https://uk.investing.com//news/coronavirus/australian-covid19-infections-set-to-hit-onemonth-low-2191503',
 'https://uk.investing.com//news/coronavirus/indias-wockhardt-to-make-covid19-vaccines-for-uk-2178023',
 'https://uk.investing.com//news/coronavirus/japan-to-agree-supply-deal-soon-for-astrazenecas-coronavirus-vaccine--kyodo-2183429',
 'https://uk.investing.com//news/stock-market-news/argentine-firm-behind-astrazeneca-covid-vaccine-latam-production-sees-aprilmay-launch-2190165',
 'https://uk.investing.com//news/stock-market-news/exdividend-trading-trips-ftse-100-travel-firm-tui-slumps-2188199',
 'https://uk.investing.com//news/stock-market-news/should-you-be-investing-money-in-pharma-stocks-2188870',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-aug-10-2184834',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-aug-13-2188144',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-aug-7-2183491',
 'https://uk.investing.com//news/coronavirus/astrazeneca-to-be-exempt-from-coronavirus-vaccine-liability-claims-in-most-countries-2175638',
 'https://uk.investing.com//news/coronavirus/moderna-vaccine-protects-against-covid19-in-monkey-study-2173551',
 'https://uk.investing.com//news/coronavirus/pfizer-biontech-to-supply-120-million-doses-of-coronavirus-vaccine-to-japan-2176539',
 'https://uk.investing.com//news/economy/biggest-obstacle-where-can-cansino-test-its-vaccine-abroad-2175203',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-tops-forecasts-on-strong-drug-sales-outlook-unchanged-2175047',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-tops-forecasts-on-strong-drug-sales-outlook-unchanged-2175047#comments',
 'https://uk.investing.com//news/stock-market-news/day-ahead-3-things-to-watch-for-july-31-2176087',
 'https://uk.investing.com//news/stock-market-news/lloyds-bank-loss-hits-ftse-100-astrazeneca-gains-2175092',
 'https://uk.investing.com//news/stock-market-news/the-astrazeneca-share-price-is-reassuringly-expensive-id-buy-it-in-a-stocks-and-shares-isa-2175489',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-235-2175930',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-july-30-2175006',
 'https://uk.investing.com//news/coronavirus/emergent-signs-174-million-deal-to-make-astrazenecas-potential-covid19-vaccine-2171528',
 'https://uk.investing.com//news/coronavirus/moderna-gets-further-472-million-us-award-for-coronavirus-vaccine-development-2171115',
 'https://uk.investing.com//news/coronavirus/moderna-pfizer-start-decisive-covid19-vaccine-trials-eye-yearend-launches-2172361',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-to-pay-up-to-6-billion-in-new-cancer-drug-deal-with-daiichi-2171295',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-lung-disease-treatment-gets-fda-nod-2170002',
 'https://uk.investing.com//news/stock-market-news/exclusive-eu-talks-with-pfizer-sanofi-jj-on-covid-vaccines-hit-snags--sources-2171875',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-24-2169987',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-27-2171271',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-28-2172442',
 'https://uk.investing.com//news/stock-market-news/us-drugmaker-earnings-heat-up-this-week-with-spotlight-on-vaccine-news-2172021',
 'https://uk.investing.com//news/coronavirus/us-government-engages-pfizer-to-produce-millions-of-doses-of-covid19-vaccine-2167948',
 'https://uk.investing.com//news/coronavirus/us-government-engages-pfizer-to-produce-millions-of-doses-of-covid19-vaccine-2167948#comments',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167330',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167336',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167337',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167338',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167339',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167340',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167341',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167342',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-22-2167701',
 'https://uk.investing.com//news/coronavirus/additional-data-on-pfizer-and-biontechs-potential-coronavirus-vaccine-shows-promise-2165897',
 'https://uk.investing.com//news/stock-market-news/asian-shares-stumble-ahead-of-eu-summit-us-stimulus-talks-2165475',
 'https://uk.investing.com//news/stock-market-news/asian-shares-stumble-ahead-of-eu-summit-us-stimulus-talks-2165475#comments',
 'https://uk.investing.com//news/stock-market-news/oil-stocks-drag-ftse-lower-ms-hit-by-job-cuts-report-2165556',
 'https://uk.investing.com//news/stock-market-news/stockbeat-astrazeneca-philips-synairgen-put-virus-fight-center-stage-2165623',
 'https://uk.investing.com//news/stock-market-news/stockbeat-astrazeneca-philips-synairgen-put-virus-fight-center-stage-2165623#comments',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167331',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167332',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167333',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167334',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-2167335',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-21-2166568',
 'https://uk.investing.com//news/coronavirus/britain-nears-500-million-pound-supply-deal-for-sanofigsk-covid19-vaccine--sunday-times-2155360',
 'https://uk.investing.com//news/coronavirus/britain-nears-500-million-pound-supply-deal-for-sanofigsk-covid19-vaccine--sunday-times-2155360#comments',
 'https://uk.investing.com//news/coronavirus/britain-signs-deals-with-pfizer-biontech-valneva-for-covid19-vaccines-2165504',
 'https://uk.investing.com//news/coronavirus/earlystage-trial-data-on-astrazeneca-covid19-vaccine-due-july-20--lancet-2163083',
 'https://uk.investing.com//news/coronavirus/emergent-signs-5yr-deal-backing-jj-covid19-vaccine-2155870',
 'https://uk.investing.com//news/coronavirus/exclusive-eu-reaches-deals-with-roche-merck-for-supply-of-covid19-drugs--source-2157475',
 'https://uk.investing.com//news/coronavirus/exclusive-eu-reaches-deals-with-roche-merck-for-supply-of-covid19-drugs--source-2157475#comments',
 'https://uk.investing.com//news/stock-market-news/stocks--europe-lower-amid-doubts-over-summit-philips-gains-2165580',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-061-2164887',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-20-2165510',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-july-8-2157331',
 'https://uk.investing.com//news/world-news/vaccine-alliance-says-75-countries-keen-to-join-covax-access-facility-2162970',
 'https://uk.investing.com//news/coronavirus/astrazeneca-taps-scottish-firm-for-tenth-covid19-vaccine-supply-deal-2148209',
 'https://uk.investing.com//news/coronavirus/exclusive-eu-in-advanced-talks-with-johnson--johnson-on-covid19-vaccine-deal--sources-2144027',
 'https://uk.investing.com//news/coronavirus/pig-trial-of-astrazenecas-covid19-vaccine-shows-promise-with-two-shots-2147045',
 'https://uk.investing.com//news/coronavirus/prince-william-visits-oxford-covid19-vaccine-developers-2148587',
 'https://uk.investing.com//news/stock-market-news/daiichi-sankyo-in-talks-with-astrazeneca-on-virus-vaccine-supply-in-japan-2149771',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-29-2150937',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-30-2151883',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-30-2151883#comments',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-june-25-2148737',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-june-26-2149661',
 'https://uk.investing.com//news/stock-market-news/vaccine-makers-face-biggest-medical-manufacturing-feat-in-history-2148966',
 'https://uk.investing.com//news/coronavirus/astrazeneca-covid19-vaccine-likely-to-protect-for-a-year--ceo-2142067',
 'https://uk.investing.com//news/coronavirus/britains-astrazeneca-ramps-up-studies-for-covid19-antibody-treatments-2136944',
 'https://uk.investing.com//news/coronavirus/britains-astrazeneca-ramps-up-studies-for-covid19-antibody-treatments-2136944#comments',
 'https://uk.investing.com//news/coronavirus/singapore-scientists-to-start-human-trials-of-covid19-vaccine-in-august-2142090',
 'https://uk.investing.com//news/coronavirus/uks-cobra-biologics-signs-astrazeneca-deal-to-supply-covid19-vaccine-2142113',
 'https://uk.investing.com//news/economy/france-gives-its-drugs-industry-a-shot-in-the-arm-to-tackle-pandemic-2142331',
 'https://uk.investing.com//news/economy/italy-germany-france-and-netherlands-sign-contract-with-astrazeneca-for-covid-vaccine-2140739',
 'https://uk.investing.com//news/economy/italy-germany-france-and-netherlands-sign-contract-with-astrazeneca-for-covid-vaccine-2140739#comments',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-shares-down-2-after-report-it-approached-gilead-over-tieup-2135997',
 'https://uk.investing.com//news/stock-market-news/uk-stocks--factors-to-watch-on-june-9-2136840',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-10-2137847',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-15-2141045',
 'https://uk.investing.com//news/coronavirus/astrazeneca-eyes-over-two-billion-doses-of-potential-covid19-vaccine-after-deals-2134276',
 'https://uk.investing.com//news/coronavirus/oxford-biomedica-inks-agreement-to-boost-uk-supply-of-astrazenecas-covid19-vaccine-2135975',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-approaches-gilead-about-potential-merger--bloomberg-news-2135748',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-targets-two-billion-doses-poor-countries-with-covid-vaccine-deals-2134383',
 'https://uk.investing.com//news/stock-market-news/ftse-100-retreats-after-rallying-on-recovery-optimism-2133775',
 'https://uk.investing.com//news/stock-market-news/londons-ftse-100-falls-as-astrazeneca-drops-on-megamerger-report-2136008',
 'https://uk.investing.com//news/stock-market-news/londons-ftse-100-falls-as-astrazeneca-drops-on-megamerger-report-2136008#comments',
 'https://uk.investing.com//news/stock-market-news/stocks--europe-slips-consolidating-after-gloomy-german-data-2136050',
 'https://uk.investing.com//news/stock-market-news/stocks--europe-slips-consolidating-after-gloomy-german-data-2136050#comments',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-5-2134706',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-june-8-2135962',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-june-8-2135988',
 'https://uk.investing.com//news/coronavirus/brazil-to-start-testing-oxford-vaccine-against-the-coronavirus-this-month-2133293',
 'https://uk.investing.com//news/coronavirus/eu-to-use-27-billion-fund-to-buy-promising-covid19-vaccines-2133828',
 'https://uk.investing.com//news/coronavirus/oxford-biomedica-inks-deal-to-make-astrazenecas-potential-covid19-vaccine-2128964',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-scores-twin-drug-approvals-alongside-vaccine-progress-2131163',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-scores-twin-drug-approvals-alongside-vaccine-progress-2131163#comments',
 'https://uk.investing.com//news/stock-market-news/london-stocks-jump-as-retailers-set-to-reopen-from-shutdown-2130865',
 'https://uk.investing.com//news/stock-market-news/london-stocks-jump-as-retailers-set-to-reopen-from-shutdown-2130865#comments',
 'https://uk.investing.com//news/stock-market-news/oxford-biomedica-hires-french-pharma-veteran-after-signing-vaccine-deal-2130853',
 'https://uk.investing.com//news/stock-market-news/recovery-hopes-power-uk-stocks-easyjet-jumps-2128718',
 'https://uk.investing.com//news/stock-market-news/recovery-hopes-power-uk-stocks-easyjet-jumps-2128718#comments',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-may-28-2128661',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-may-29-2129625',
 'https://uk.investing.com//news/stock-market-news/uk-stocksfactors-to-watch-on-may-29-2129625#comments',
 'https://uk.investing.com//news/stock-market-news/update-1uk-stocksfactors-to-watch-on-may-29-2129688',
 'https://uk.investing.com//news/coronavirus/how-one-indian-company-could-be-worlds-door-to-a-covid19-vaccine-2125675',
 'https://uk.investing.com//news/coronavirus/keenlywatched-covid19-vaccine-wont-be-expensive-developer-says-2120374',
 'https://uk.investing.com//news/coronavirus/london-stocks-dip-as-coronavirus-damage-mounts-2112566',
 'https://uk.investing.com//news/coronavirus/project-leader-oxfords-covid19-vaccine-trial-has-50-chance-of-success--telegraph-2126063',
 'https://uk.investing.com//news/coronavirus/take-five-coronavirus-vaccine-race-is-on-2125552',
 'https://uk.investing.com//news/economy/top-5-things-to-know-in-the-market-on-thursday-may-21st-2124376',
 'https://uk.investing.com//news/economy/top-5-things-to-know-in-the-market-on-thursday-may-21st-2124376#comments',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-in-talks-with-other-governments-to-replicate-uk-vaccine-deal-2121912',
 'https://uk.investing.com//news/stock-market-news/here-are-my-top-3-investing-ideas-of-the-decade-for-navigating-the-ftse-100-stock-index-2121300',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-123-2127339',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-134-2128318',
 'https://uk.investing.com//news/coronavirus/astrazeneca-to-produce-oxford-universitys-potential-covid19-vaccine-2108103',
 'https://uk.investing.com//news/coronavirus/astrazeneca-trials-farxiga-as-potential-treatment-for-covid19-patients-2102667',
 'https://uk.investing.com//news/coronavirus/pfizer-aims-for-1020-million-doses-of-coronavirus-vaccine-by-end2020-for-emergency-use-2108694',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-beats-firstquarter-profit-estimates-retains-2020-forecast-2106965',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-diabetes-drug-gets-us-nod-to-treat-heart-failure-2112601',
 'https://uk.investing.com//news/stock-market-news/ftse-100-edges-lower-amid-earnings-jitters-2099570',
 'https://uk.investing.com//news/stock-market-news/gsk-firstquarter-profit-beats-on-steady-demand-for-shingrix-2107229',
 'https://uk.investing.com//news/stock-market-news/london-stocks-gain-as-global-economies-reopen-stanchart-jumps-2107019',
 'https://uk.investing.com//news/stock-market-news/stocks--europe-edges-lower-ftse-100-outperforms-2112627',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-086-2095895',
 'https://uk.investing.com//news/coronavirus/astrazeneca-to-start-calquence-in-covid19-trial-2095229',
 'https://uk.investing.com//news/economy/top-5-things-to-know-in-the-market-on-friday-february-14th-2052237',
 'https://uk.investing.com//news/economy/top-5-things-to-know-in-the-market-on-friday-february-14th-2052237#comments',
 'https://uk.investing.com//news/stock-market-news/abbvieallergan-63-billion-deal-aided-by-nestle-astrazeneca-buys-2038380',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-rbs-results-drag-down-ftse-100-2052100',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-2020-sales-growth-to-take-coronavirus-hit-2052064',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-ceo-reaffirms-2021-targets-2052274',
 'https://uk.investing.com//news/stock-market-news/centrica-barclays-weigh-down-ftse-100-2051109',
 'https://uk.investing.com//news/stock-market-news/stocks--europe-heads-lower-rbs-renault-astrazeneca-weigh-2052116',
 'https://uk.investing.com//news/stock-market-news/stocks--europe-pushes-higher-on-virus-hopes-chinese-data-2095265',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-058-2052650',
 'https://uk.investing.com//news/forex-news/premarket-london-william-hill-beats-profit-expectations-2029433',
 'https://uk.investing.com//news/stock-market-news/astrazenecamercks-lynparza-gets-fda-okay-to-treat-pancreatic-cancer-2022427',
 'https://uk.investing.com//news/stock-market-news/defensive-stocks-buoy-ftse-100-before-uschina-trade-deal-2031182',
 'https://uk.investing.com//news/stock-market-news/defensive-stocks-buoy-ftse-100-before-uschina-trade-deal-2031182#comments',
 'https://uk.investing.com//news/stock-market-news/drugmakers-slash-prices-to-be-eligible-for-chinas-bulkbuy-program-2033620',
 'https://uk.investing.com//news/stock-market-news/ftse-100-dips-on-yearend-profit-taking-stronger-pound-2022971',
 'https://uk.investing.com//news/stock-market-news/japan-us-lead-surveys-corporate-climate-change-action-a-list-2034006',
 'https://uk.investing.com//news/stock-market-news/jpmorgan-to-buy-paris-building-to-bring-euro-teams-from-london-2033818',
 'https://uk.investing.com//news/stock-market-news/premarket-london-shell-raises-oil-output-sees-charges-forecast-2019147',
 'https://uk.investing.com//news/stock-market-news/roche-wins-approval-for-cancer-drug-kadcyla-in-fastgrowing-china-market-2035877',
 'https://uk.investing.com//news/stock-market-news/uk-stock-rally-halts-as-traders-pocket-profits-2022442',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-shares-rise-on-early-us-approval-for-leukaemia-drug-2002847',
 'https://uk.investing.com//news/stock-market-news/astrazenecamerck-lynparza-wins-approval-in-china-for-ovarian-cancer-treatment-2009413',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-imfinzi-gets-fda-priority-review-for-small-cell-lung-cancer-2006250',
 'https://uk.investing.com//news/stock-market-news/ftse-extends-slide-as-trumps-tariff-salvo-dents-sentiment-2007804',
 'https://uk.investing.com//news/stock-market-news/ftse-extends-slide-as-trumps-tariff-salvo-dents-sentiment-2007804#comments',
 'https://uk.investing.com//news/stock-market-news/ftse-falls-for-fifth-day-on-trump-trade-rhetoric-firm-pound-2008608',
 'https://uk.investing.com//news/stock-market-news/ftse-slips-on-exdiv-trading-uschina-trade-doubts-2005931',
 'https://uk.investing.com//news/stock-market-news/premarket-london-boohoo-reports-record-black-friday-ferguson-solid-2007781',
 'https://uk.investing.com//news/stock-market-news/premarket-london-ds-smith-posts-strong-h1-ig-revenue-flat-2009392',
 'https://uk.investing.com//news/stock-market-news/uk-shares-bounce-back-after-tradedriven-losses-2002586',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-048-2018851',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-raises-sales-forecast-after-surge-in-cancer-drugs-1984507',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sells-european-rights-for-schizophrenia-drug-to-cheplapharm-1987915',
 'https://uk.investing.com//news/stock-market-news/britains-astrazeneca-launches-1-billion-china-investment-fund-with-cicc-1992036',
 'https://uk.investing.com//news/stock-market-news/premarket-london-ba-parent-cuts-growth-estimates-astrazeneca-success-1994278',
 'https://uk.investing.com//news/stock-market-news/premarket-london-talk-talk-swings-to-1h-profit-888-defiant-1998659',
 'https://uk.investing.com//news/stock-market-news/rising-hong-kong-tensions-drag-asiaexposed-uk-stocks-1995494',
 'https://uk.investing.com//news/stock-market-news/the-elephant-in-the-room-with-astrazenecas-latest-update-1986227',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-039-1988353',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-105-1984868',
 'https://uk.investing.com//news/stock-market-news/what-next-for-the-astrazeneca-share-price-1987373',
 'https://uk.investing.com//news/forex-news/newsbreak-astrazenecas-diabetes--drug-farxiga-gets-us-approval-1981809',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-strengthens-lynparza-push-with-prostate-cancer-data-1972711',
 'https://uk.investing.com//news/stock-market-news/factbox--how-are-businesses-preparing-for-a-nodeal-brexit-1979126',
 'https://uk.investing.com//news/stock-market-news/ftse-100-on-track-for-fourth-session-of-gains-1984363',
 'https://uk.investing.com//news/stock-market-news/ftse-gains-as-thomas-cook-collapse-buoys-bluechip-rivals-1967572',
 'https://uk.investing.com//news/stock-market-news/ftse-gains-as-thomas-cook-collapse-buoys-bluechip-rivals-1967572#comments',
 'https://uk.investing.com//news/stock-market-news/premarket-london-astrazeneca-raises-guidance-ppi-drives-rbs-to-loss-1984325',
 'https://uk.investing.com//news/stock-market-news/stockbeat-dax-hits-17month-high-on-hope-that-germany-has-hit-bottom-1984451',
 'https://uk.investing.com//news/stock-market-news/stockbeat-dax-hits-17month-high-on-hope-that-germany-has-hit-bottom-1984451#comments',
 'https://uk.investing.com//news/stock-market-news/uk-shares-climb-higher-ferguson-advances-1973151',
 'https://uk.investing.com//news/stock-market-news/uk-shares-climb-higher-ferguson-advances-1973151#comments',
 'https://uk.investing.com//news/stock-market-news/us-fda-approves-astrazeneca-diabetes-drug-for-treating-heart-failure-risk-1981828',
 'https://uk.investing.com//news/stock-market-news/weak-us-euro-zone-data-push-european-shares-lower-1973694',
 'https://uk.investing.com//news/economic-indicators/jd-sports-leads-ftse-100-higher-rate-cut-hopes-support-1959515',
 'https://uk.investing.com//news/politics/with-a-backup-to-the-backup-insulin-makers-say-theyre-primed-for-brexit-1963697',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-tagrisso-gets-china-ok-for-type-of-lung-cancer-1955022',
 'https://uk.investing.com//news/stock-market-news/ftse-100-little-changed-as-several-stocks-trade-exdividend-industrials-rise-1956064',
 'https://uk.investing.com//news/stock-market-news/ftse-100-slips-as-pound-recovers-diageo-slides-1961790',
 'https://uk.investing.com//news/stock-market-news/londons-bluechip-index-gains-ground-sirius-minerals-sinks-1963709',
 'https://uk.investing.com//news/stock-market-news/londons-bluechip-index-gains-ground-sirius-minerals-sinks-1963709#comments',
 'https://uk.investing.com//news/stock-market-news/steady-pound-growth-worries-pressure-ftse-100-galliford-jumps-1958965',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-064-1966187',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-111-1953890',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-069-1958588',
 'https://uk.investing.com//news/economy/miners-help-ftse-100-override-trade-worries-1953652',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-lupus-drug-meets-main-goal-in-study-1951715',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-says-us-fda-grants-fast-track-status-for-diabetes-drug-farxiga-1949931',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-farxiga-meets-main-goal-of-heart-failure-study-1945453',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-imfinzi-misses-main-goal-of-advanced-lung-cancer-study-1946341',
 'https://uk.investing.com//news/stock-market-news/encouraging-china-trade-comments-weaker-pound-lift-ftse-100-1951738',
 'https://uk.investing.com//news/stock-market-news/european-shares-rise-for-third-straight-session-1945457',
 'https://uk.investing.com//news/stock-market-news/london-leads-european-markets-lower-as-investors-play-waiting-game-1949955',
 'https://uk.investing.com//news/stock-market-news/should-i-sell-my-shares-if-a-recession-is-imminent-1948830',
 'https://uk.investing.com//news/stock-market-news/weak-pound-astrazeneca-push-ftse-higher-1945464',
 'https://uk.investing.com//news/economy/glencore-fall-hits-mining-stocks-as-ftse-steadies-1935606',
 'https://uk.investing.com//news/stock-market-news/as-steep-patent-cliff-looms-shionogi-moves-to-develop-its-own-us-sales-staff-1918220',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-quarterly-product-sales-beat-on-cancer-drug-strength-1924889',
 'https://uk.investing.com//news/stock-market-news/exclusive-woodford-biotech-stakes-first-off-block-in-asset-sale--source-1918802',
 'https://uk.investing.com//news/stock-market-news/ftse-100-flat-after-mixed-corporate-earnings-1924927',
 'https://uk.investing.com//news/stock-market-news/ftse-100-hits-10month-high-on-weaker-sterling-telford-soars-1909535',
 'https://uk.investing.com//news/stock-market-news/ftse-100-set-to-snap-sixday-losing-streak-thomas-cook-plummets-1915366',
 'https://uk.investing.com//news/stock-market-news/gsk-forecasts-smaller-hit-to-fullyear-profit-after-second-quarter-beat-1923817',
 'https://uk.investing.com//news/stock-market-news/gsk-names-hsbcs-symonds-as-nonexec-chairman-1923666',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-019-1925693',
 'https://uk.investing.com//news/economy-news/top-5-things-to-know-in-the-market-on-thursday-1446697',
 'https://uk.investing.com//news/stock-market-news/ftse-breaks-threeday-losing-streak-vodafone-hits-multiyear-lows-1431799',
 'https://uk.investing.com//news/stock-market-news/heres-a-dirt-cheap-way-of-creating-a-second-income-stream-through-the-stock-market-1432647',
 'https://uk.investing.com//news/stock-market-news/italy-drugmaker-zambon-ready-to-splash-out-in-us-china-push-1431073',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-065-1431204',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-085-1432273',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-051-1430029',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-034-1447169',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-058-1477823',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-143-1517876',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-picks-baselga-to-lead-oncology-rd-in-growth-plan-1418928',
 'https://uk.investing.com//news/stock-market-news/british-shares-on-track-for-best-week-since-september-housebuilders-bounce-1422360',
 'https://uk.investing.com//news/stock-market-news/ftse-flat-ahead-of-trade-talks-dunelm-lifts-midcaps-1418636',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-032-1419104',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-089-1421141',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-122-1420238',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-040-1427987',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-050-1423730',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-039-1422651',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-096-1423744',
 'https://uk.investing.com//news/economy-news/pharma-industry-returns-on-rd-investment-hit-9year-low-1408920',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-imfinzi-fails-to-meet-main-goals-in-head-and-neck-cancer-study-1401087',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-ovarian-cancer-and-anaemia-treatments-meet-goals-in-latestage-studies-1409953',
 'https://uk.investing.com//news/stock-market-news/ftse-100-tumbles-as-weak-chinese-data-dents-markets-1406011',
 'https://uk.investing.com//news/stock-market-news/oil-drives-uk-stocks-up-as-relief-bounce-spills-over-from-us-1413641',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-197-1404630',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-088-1416880',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-214-1410439',
 'https://uk.investing.com//news/stock-market-news/uk-markets-rise-as-the-curtain-falls-on-a-dim-2018-1414670',
 'https://uk.investing.com//news/stock-market-news/uk-stocks-climb-wall-street-ladder-1412958',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-returns-to-sales-growth-led-by-cancer-medicines-1373277',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sells-older-asthma-rhinitis-drugs-for-350-million-1369775',
 'https://uk.investing.com//news/stock-market-news/ftse-outperforms-european-peers-as-brexit-angst-sinks-sterling-1377046',
 'https://uk.investing.com//news/stock-market-news/ftse-supported-by-financials-astrazeneca-1373358',
 'https://uk.investing.com//news/stock-market-news/micro-focus-leads-ftse-higher-in-choppy-trade-1368346',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-065-1383565',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-012-1374126',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-029-1370547',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-104-1377654',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-044-1374155',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sells-some-rights-to-nexium-vimovo-to-grunenthal-1360905',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-to-buy-98-percent-stake-in-frances-innate-pharma-1352344',
 'https://uk.investing.com//news/stock-market-news/roche-posts-modest-third-quarter-sales-beat-as-new-drugs-offset-biosimilar-hit-1346077',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-030-1356493',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-038-1346932',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-046-1365459',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-080-1368912',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-015-1368941',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-019-1365485',
 'https://uk.investing.com//news/stock-market-news/why-i-think-it-might-finally-be-time-to-buy-ftse-100-dividend-stock-glaxosmithkline-1363574',
 'https://uk.investing.com//news/politics-news/astrazeneca-halts-uk-investments-due-to-brexit-uncertainties--le-monde-1343686',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-plots-china-robot-offensive-to-counter-price-cuts-1319833',
 'https://uk.investing.com//news/stock-market-news/pfizer-ceo-ian-read-to-step-down-next-year-1332128',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-010-1326555',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-010-1328147',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-019-1314675',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-031-1332684',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-109-1329560',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-084-1330834',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-090-1342012',
 'https://uk.investing.com//news/politics-news/french-drugmakers-urge-brexit-negotiators-to-safeguard-supply-chain-1307223',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-gets-fda-breakthrough-therapy-label-for-asthma-treatment-1307968',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-switches-contractor-as-new-headquarter-delayed-again-1292654',
 'https://uk.investing.com//news/stock-market-news/pfizer-bets-on-biotech-flu-vaccine-in-425-million-biontech-alliance-1286782',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-051-1290208',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-053-1279710',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-111-1287551',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-010-1313152',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-031-1283104',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-088-1281210',
 'https://uk.investing.com//news/politics-news/sanofi-has-been-stockpiling-drugs-for-nodeal-brexit--wsj-1267712',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-key-lung-cancer-drug-wins-european-panel-thumbsup-1262958',
 'https://uk.investing.com//news/stock-market-news/ftse-extends-gains-as-oil-stocks-power-ahead-housebuilders-fall-1277198',
 'https://uk.investing.com//news/stock-market-news/shell-slide-weighs-on-ftse-as-earnings-in-focus-1260709',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-011-1273978',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-028-1267175',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-063-1275564',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-071-1276169',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-098-1261544',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-011-1261592',
 'https://uk.investing.com//news/politics-news/astrazeneca-to-stockpile-drugs-as-brexit-safety-net-1250163',
 'https://uk.investing.com//news/politics-news/eu-drug-regulators-step-up-work-to-prepare-for-no-deal-brexit-1253813',
 'https://uk.investing.com//news/stock-market-news/emirates-starts-london-stansted-flights-as-rival-airports-near-capacity-1208604',
 'https://uk.investing.com//news/stock-market-news/european-shares-rise-as-useu-trade-breakthrough-spurs-carmakers-1260634',
 'https://uk.investing.com//news/stock-market-news/ftse-100-recovers-driven-by-defensives-as-trade-nerves-ebb-1243683',
 'https://uk.investing.com//news/stock-market-news/gsks-new-rd-head-bets-on-genetics-with-300-million-23andme-deal-1258822',
 'https://uk.investing.com//news/stock-market-news/hitting-cancer-early--astrazenecas-bid-to-outmanoeuvre-rivals-1237658',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-100-1244438',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-079-1244457',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-171-1224217',
 'https://uk.investing.com//news/politics-news/eu-industry-lobby-adds-to-pressure-on-britain-over-brexit-talks-1191553',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-cancer-drug-hits-second-goal-by-extending-survival-1186690',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-drug-fasenra-fails-to-achieve-main-goal-in-copd-trial-1190744',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-potassium-drug-finally-approved-threatening-vifor-1180234',
 'https://uk.investing.com//news/stock-market-news/ftse-rebounds-kingfisher-jumps-as-homebase-changes-hands-1186783',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-029-1180799',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-126-1200957',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-030-1202354',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-138-1184152',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-010-1178913',
 'https://uk.investing.com//news/politics-news/mps-call-for-postbrexit-pharmaceutical-deal-with-eu-1176318',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-hit-by-falling-crestor-sales-higher-costs-1178131',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sells-rights-for-seroquel-to-luye-pharma-for-538-million-1162883',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-to-raise-stake-in-circassia-to-up-to-199-percent-1140263',
 'https://uk.investing.com//news/stock-market-news/bp-oil-prices-keep-ftse-afloat-1152095',
 'https://uk.investing.com//news/stock-market-news/experts-query-case-for-glaxosmithklines-new-3in1-lung-drug-1133299',
 'https://uk.investing.com//news/stock-market-news/gsk-bets-on-lift-from-new-lung-drugs-ahead-of-advairs-last-gasp-1177974',
 'https://uk.investing.com//news/stock-market-news/industry-fears-disruption-as-eu-excludes-uk-from-drug-approvals-1132280',
 'https://uk.investing.com//news/stock-market-news/shires-mega-deal-fails-to-lift-ftse-amid-global-gloom-1142725',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-016-1178892',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sees-pivotal-lung-cancer-trial-results-later-in-2018-1075109',
 'https://uk.investing.com//news/stock-market-news/compugen-astrazeneca-unit-in-cancer-drug-development-deal-1107229',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-013-1102622',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-045-1079711',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-124-1130541',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-003-1128516',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-118-1083194',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-166-1085413',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-173-1109632',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-219-1090707',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-immunotherapy-drug-wins-key-lung-cancer-approval-1044342',
 'https://uk.investing.com//news/stock-market-news/big-pharma-big-data--why-drugmakers-want-your-health-records-1060631',
 'https://uk.investing.com//news/stock-market-news/ftse-100-dips-as-strong-pound-dampens-dollarearners-1032837',
 'https://uk.investing.com//news/stock-market-news/gains-for-south-africaexposed-stocks-cyclicals-soothe-ftse-100-1038392',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-026-1073706',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-069-1039740',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-078-1067867',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-129-1028293',
 'https://uk.investing.com//news/stock-market-news/us-fda-approves-astrazenecas-immunotherapy-for-lung-cancer-1043159',
 'https://uk.investing.com//news/stock-market-news/us-health-officials-reverse-stance-on-astrazenecas-flu-vaccine-1048757',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-flags-return-to-drug-sales-growth-in-2018-1012061',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-links-with-alibaba-and-tencent-in-china-push-1012291',
 'https://uk.investing.com//news/stock-market-news/deutsche-bank-drags-european-shares-into-worst-week-since-august-1012111',
 'https://uk.investing.com//news/stock-market-news/ftse-100-heads-for-worst-week-in-six-months-as-results-drag-1012179',
 'https://uk.investing.com//news/stock-market-news/ftse-dips-ahead-of-boe-meeting-but-drugmakers-give-support-1023908',
 'https://uk.investing.com//news/stock-market-news/gsk-cautious-on-2018-as-drug-competition-looms-after-fourthquarter-beats-1021046',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-106-1013420',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-202-1025519',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-062-1013437',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-146-1016463',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-is-sued-by-array-biopharma-over-cancer-drug-royalties-1011610',
 'https://uk.investing.com//news/stock-market-news/astrazenecas-inhaler-for-lung-disease-shows-improved-function-in-late-stage-trail-998281',
 'https://uk.investing.com//news/stock-market-news/at-astrazeneca-fewer-drug-projects-bring-big-productivity-jump-984564',
 'https://uk.investing.com//news/stock-market-news/new-drug-approvals-hit-21year-high-in-2017-954222',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-101-959578',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-018-969510',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-073-952099',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-007-950424',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-064-952155',
 'https://uk.investing.com//news/stock-market-news/uk-stocks-end-two-days-of-currencydriven-declines-998361',
 'https://uk.investing.com//news/stock-market-news/at-astrazeneca-fewer-drug-projects-bring-big-productivity-jump-984564',
 'https://uk.investing.com//news/stock-market-news/new-drug-approvals-hit-21year-high-in-2017-954222',
 'https://uk.investing.com//news/stock-market-news/over-20-percent-of-uk-companies-faced-shareholder-revolt--pay-study-935547',
 'https://uk.investing.com//news/stock-market-news/roche-to-seize-leapfrog-opportunity-in-lung-cancer-847941',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-101-959578',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-018-969510',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-041-950386',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-073-952099',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-007-950424',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-064-952155',
 'https://uk.investing.com//news/politics-news/brexit-gets-real-for-drugmakers-as-regulator-moves-to-amsterdam-749419',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-thirdquarter-sales-weak-as-it-waits-for-medicine-to-work-724059',
 'https://uk.investing.com//news/stock-market-news/european-shares-open-a-touch-lower-as-more-thirdquarter-earnings-pour-in-724070',
 'https://uk.investing.com//news/stock-market-news/ftse-rises-as-sterlings-decline-boosts-exporters-727921',
 'https://uk.investing.com//news/stock-market-news/ftse-stuck-at-sevenweek-lows-as-cyclicals-shire-declines-745186',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-009-736557',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-012-733086',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-021-803843',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-025-729638',
 'https://uk.investing.com//news/stock-market-news/uk-shares-lower-at-close-of-trade-investingcom-united-kingdom-100-down-094-803908',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-asthma-drug-tralokinumab-disappoints-again-650228',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-licenses-genetic-disease-drug-to-mereo-biopharma-649745',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-plans-new-pivotal-lung-cancer-trial-with-incyte-649996',
 'https://uk.investing.com//news/stock-market-news/former-gsk-boss-to-lead-new-uk-accelerated-drug-access-scheme-650695',
 'https://uk.investing.com//news/stock-market-news/gsk-nabs-calico-scientist-barron-as-new-rd-head-vallance-quits-723895',
 'https://uk.investing.com//news/stock-market-news/gsks-rd-head-to-leave-for-top-uk-government-job--source-650732',
 'https://uk.investing.com//news/stock-market-news/pfizer-consumer-units-sales-fall-ahead-of-divestiture-call-650142',
 'https://uk.investing.com//news/stock-market-news/short-seller-kerrisdale-targets-woodfordbacked-prothena-723921',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-004-650625',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-035-723701',
 'https://uk.investing.com//news/politics-news/drugs-firm-astrazeneca-wants-threeyear-brexit-transition-515906',
 'https://uk.investing.com//news/politics-news/pharma-and-medtech-industries-prepare-for-hard-brexit-518490',
 'https://uk.investing.com//news/politics-news/tell-us-how-brexit-will-work-british-companies-demand-515920',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-among-backers-as-swiss-cancer-biotech-raises-200-million-519987',
 'https://uk.investing.com//news/stock-market-news/china-biotechs-coming-out-party-masks-long-road-ahead-519163',
 'https://uk.investing.com//news/stock-market-news/drug-industry-fails-to-win-judicial-review-of-new-uk-cost-curbs-516367',
 'https://uk.investing.com//news/stock-market-news/ftse-scales-twomonth-peak-516743',
 'https://uk.investing.com//news/stock-market-news/merck-posts-quarterly-loss-versus-yearago-profit-521046',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-017-515950',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-021-514696',
 'https://uk.investing.com//news/stock-market-news/aspen-to-buy-remaining-astrazeneca-anaesthetic-rights-for-up-to-766-million-512669',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-rebuilds-cancer-drug-hopes-with-new-lung-data-511749',
 'https://uk.investing.com//news/stock-market-news/gates-foundation-invests-40-million-in-uk-immunotherapy-company-513297',
 'https://uk.investing.com//news/stock-market-news/insurers-lead-european-shares-higher-after-hurricane-irma-downgrade-511951',
 'https://uk.investing.com//news/stock-market-news/mercks-immunotherapy-chemo-mix-shows-extended-lung-cancer-gains-511522',
 'https://uk.investing.com//news/stock-market-news/sterling-high-hurts-ftse-100-512448',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-010-513619',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-028-514325',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-128-512041',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-068-514328',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sought-to-buy-japans-daiichi-sankyo-last-year--report-510015',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-taps-ai-for-drug-discovery-in-deal-with-berg-509518',
 'https://uk.investing.com//news/stock-market-news/daiichi-sankyo-denies-receiving-takeover-bid-from-astrazeneca-510143',
 'https://uk.investing.com//news/stock-market-news/ftse-100-flat-as-wpp-sinks-508670',
 'https://uk.investing.com//news/stock-market-news/new-astrazeneca-amgen-biotech-drug-offers-broad-asthma-relief-511258',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-076-511455',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-077-510386',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade-omx-stockholm-30-up-099-509915',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-037-511183',
 'https://uk.investing.com//news/stock-market-news/uk-shares-higher-at-close-of-trade-investingcom-united-kingdom-100-up-067-511460',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-investor-woodford-says-share-fall-on-mystic-results-not-justified-504666',
 'https://uk.investing.com//news/stock-market-news/cyclicals-housebuilders-dent-ftse-coca-cola-hbc-rises-506695',
 'https://uk.investing.com//news/stock-market-news/dividends-a-worry-at-drugmakers-astra,-gsk-after-dramatic-week-504682',
 'https://uk.investing.com//news/stock-market-news/fda-approves-expanded-use-of-astrazeneca-ovarian-cancer-drug-507900',
 'https://uk.investing.com//news/stock-market-news/ftse-heads-for-weekly-loss-as-italy-takes-toll-on-bt-504669',
 'https://uk.investing.com//news/stock-market-news/income-investors-run-for-cover-as-dependable-uk-dividends-in-jeopardy-506518',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-078-507852',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade-omx-stockholm-30-down-105-506774',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.82-504751',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-1.03-504755',
 'https://uk.investing.com//news/economy-news/top-5-things-to-know-in-the-market-on-thursday-504431',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-drug-sales-slide-as-key-cancer-drug-news-awaited-504358',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-pain-weighs-on-ftse,-diageo-makes-merry-after-results-504410',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-slump-dominates-european-open-on-busy-earnings-day-504389',
 'https://uk.investing.com//news/stock-market-news/bayer-cuts-profit-forecast-after-consumer-health-headache-504450',
 "https://uk.investing.com//news/stock-market-news/glaxosmithkline's-new-boss-streamlines-drug-research-504174",
 'https://uk.investing.com//news/stock-market-news/global-stocks-mostly-higher-as-fed-holds,-heavy-earnings-flow-504406',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.62-504526',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-0.08-504532',
 'https://uk.investing.com//news/stock-market-news/u.k.-stocks-edge-lower-after-earnings-spate,-astrazeneca-plunges-504428',
 "https://uk.investing.com//news/stock-market-news/astrazeneca's-ceo-soriot-to-join-israeli-drugs-company-teva---report-501658",
 "https://uk.investing.com//news/stock-market-news/astrazeneca-holds-ftse-back,-carillion-loses-last-'buy'-rating-501712",
 'https://uk.investing.com//news/stock-market-news/astrazeneca-offers-no-comment-after-report-saying-ceo-staying-502093',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-shares-fall-for-a-second-day-over-ceo-uncertainty-501945',
 'https://uk.investing.com//news/stock-market-news/ftse-falters,-but-set-for-weekly-gain;-carillion-sees-relief-501963',
 "https://uk.investing.com//news/stock-market-news/glaxosmithkline's-new-ceo-prepares-to-trim-drug-pipeline-503341",
 'https://uk.investing.com//news/stock-market-news/gsk-hires-ex-wal-mart-executive-to-transform-tech-operations-503941',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.49-501820',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-0.08-501825',
 'https://uk.investing.com//news/stock-market-news/u.k.-stocks-edge-up-as-pound-firms-501735',
 'https://uk.investing.com//news/politics-news/british-ministers-want-postbrexit-drug-regulation-deal-with-eu-499704',
 'https://uk.investing.com//news/politics-news/ftse-haunted-by-uncertainty-one-year-after-brexit-vote-497792',
 'https://uk.investing.com//news/stock-market-news/astrazeneca,-chi-med-take-kidney-cancer-drug-into-final-testing-498899',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-cancer-drug-trial-prompts-investors-to-take-options-cover-498536',
 'https://uk.investing.com//news/stock-market-news/ftse-down-in-the-dumps-as-defensives,-pearson-and-m-amp;s-fall-501228',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.15-498751',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.11-498285',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.20-501310',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.38-500416',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.91-498513',
 'https://uk.investing.com//news/stock-market-news/financials-lift-ftse-as-markets-brace-for-election-178172',
 'https://uk.investing.com//news/stock-market-news/ftse-edge-up-helped-by-m-amp;s-but-miners-weigh-175317',
 'https://uk.investing.com//news/stock-market-news/ftse-wallows-at-one-week-low-as-energy-weighs;-imagination-tech-rockets-497521',
 'https://uk.investing.com//news/stock-market-news/gsk-to-get-new-pharma-head-in-september,-after-astrazeneca-tussle-496810',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.22-497609',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.60-179723',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.67-179011',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.35-497078',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.55-179992',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-0.76-497081',
 "https://uk.investing.com//news/stock-market-news/astrazeneca's-durvalumab-shown-in-trial-to-reduce-risk-of-death-from-lung-cancer-172791",
 'https://uk.investing.com//news/stock-market-news/ftse-heads-for-third-straight-week-of-gains-as-astrazeneca-rises-172840',
 'https://uk.investing.com//news/stock-market-news/how-astrazeneca-cancer-drug-forecasts-are-set-to-rise-173325',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.27-172949',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.83-172069',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.86-174465',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.21-172349',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.58-172077',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.62-172953',
 'https://uk.investing.com//news/stock-market-news/u.k.-stocks-edge-higher,-pound-lower-as-boe-holds-172827',
 'https://uk.investing.com//news/economy-news/circassia-gives-up-on-allergy-after-house-dust-mite-study-fails-167868',
 'https://uk.investing.com//news/politics-news/brexit-doesnt-mean-tight-immigration-minister-tells-scientists-165554',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-boosts-early-respiratory-medicine-with-pieris-deal-170764',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-moving-costs-rise-as-new-headquarter-nears-completion-169280',
 'https://uk.investing.com//news/stock-market-news/ftse-hits-one-month-low-as-miners,-babcock-slide-163660',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.18-166652',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.35-166060',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-2.29-169081',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.17-165804',
 'https://uk.investing.com//news/stock-market-news/weak-sterling-boosts-glaxsmithkline-as-new-ceo-takes-over-169521',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-ovarian-cancer-drug-slows-disease-markedly-in-study-161041',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-wins-approval-for-lung-cancer-pill-in-china-163618',
 'https://uk.investing.com//news/stock-market-news/ftse-slips-from-record-high-as-banks,-oil-drag-162161',
 'https://uk.investing.com//news/stock-market-news/gsk-and-regeneron-to-mine-gene-data-from-500,000-britons-162871',
 'https://uk.investing.com//news/stock-market-news/miners-support-ftse,-m-amp;a-sends-mid-caps-to-record-high-160695',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.16-159903',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.17-163062',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.33-163756',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.50-162794',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.68-158789',
 "https://uk.investing.com//news/stock-market-news/astrazeneca's-lynparza-meets-main-goal-in-breast-cancer-study-156057",
 'https://uk.investing.com//news/stock-market-news/european-shares-set-for-second-weekly-gain;-allianz-rallies-after-update-155976',
 'https://uk.investing.com//news/stock-market-news/ftse-retreats-from-one-month-peak,-commodity-linked-stocks-weigh-155757',
 'https://uk.investing.com//news/stock-market-news/ftse-set-for-week-of-modest-gains,-pharmas-lead-156016',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.15-155860',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.39-157299',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.35-158480',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.36-156100',
 'https://uk.investing.com//news/stock-market-news/top-astrazeneca-shareholder-woodford-adds-to-stake,-bullish-on-outlook-155853',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.03-155865',
 'https://uk.investing.com//news/politics-news/danish-drugmaker-novo-nordisk-bets-115-million-pounds-on-postbrexit-uk-science-152048',
 'https://uk.investing.com//news/politics-news/drugmakers-look-for-reciprocal-ukeu-regulation-after-brexit-152972',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-faces-falling-2017-profits-as-key-drug-data-awaited-152837',
 'https://uk.investing.com//news/stock-market-news/new-glaxosmithkline-ceo-prepares-for-r-d-choices,-potential-deals-153595',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.13-153872',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.27-155601',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.35-153264',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.51-152433',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.53-154388',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.54-153648',
 "https://uk.investing.com//news/stock-market-news/credit-suisse,-astrazeneca-among-top-bofa-ml's-top-european-picks-for-first-quarter-146782",
 'https://uk.investing.com//news/stock-market-news/ftse-hits-record-highs,-extends-historic-winning-streak-148761',
 'https://uk.investing.com//news/stock-market-news/merck-steals-march-on-rivals-in-lung-cancer-drug-combo-race-148216',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.12-148318',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.76-150351',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.21-151032',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.08-146440',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.14-150088',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.58-147279',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-0.15-150355',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-in-broad-drug-deal-with-bicycle-worth-up-to-$1-billion-141608',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-oncology-head-jumps-ship-to-become-innate-ceo-144871',
 'https://uk.investing.com//news/stock-market-news/ftse-gains,-on-track-for-best-week-since-july-143212',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.24-143599',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.59-144372',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.79-142800',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.93-143265',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.08-143037',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.35-144907',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.73-145296',
 'https://uk.investing.com//news/stock-market-news/chinese-drugmaker-invests-$36.5-million-in-british-biotech-business-kymab-140291',
 'https://uk.investing.com//news/stock-market-news/french-cancer-specialist-innate-pharma-can-remain-independent,-for-now---ceo-140575',
 'https://uk.investing.com//news/stock-market-news/ftse-edges-lower-as-major-firms-trade-ex-divs-140283',
 'https://uk.investing.com//news/stock-market-news/ftse-looks-set-for-third-straight-week-of-gains-140492',
 'https://uk.investing.com//news/stock-market-news/glaxosmithkline-seeks-u.s.-approval-for-triple-lung-drug-139579',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.44-139903',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.46-140581',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.95-141420',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.14-140588',
 'https://uk.investing.com//news/stock-market-news/u.s.-court-upholds-astrazeneca,-ranbaxy-win-in-nexium-antitrust-trial-139733',
 "https://uk.investing.com//news/stock-market-news/astrazeneca's-drug-sales-weighed-on-by-flagging-crestor-137568",
 "https://uk.investing.com//news/stock-market-news/astrazeneca-pauses-two-cancer-drug-trials'-enrolment-due-to-bleeding-135020",
 "https://uk.investing.com//news/stock-market-news/britain's-ftse-hits-one-month-low,-stanchart-falls-again-135999",
 'https://uk.investing.com//news/stock-market-news/ftse-dips-after-trump-win-though-pharma,-gold-miners-surge-137366',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.30-139319',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.95-139080',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-1.00-135840',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-1.01-136087',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.47-135001',
 'https://uk.investing.com//news/stock-market-news/vectura-on-lookout-for-deals-to-build-specialised-lung-drug-business-135830',
 "https://uk.investing.com//news/economy-news/it's-lonely-at-the-top-for-ftse-100-index-132287",
 'https://uk.investing.com//news/stock-market-news/astrazeneca-cuts-brilinta-sales-hopes-on-artery-disease-failure-130492',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-licenses-respiratory-drug-to-insmed-130685',
 'https://uk.investing.com//news/stock-market-news/glaxosmithkline-to-pay-$20-million-to-settle-foreign-bribery-related-charges-129950',
 'https://uk.investing.com//news/stock-market-news/hard-to-swallow---emerging-markets-get-tougher-for-drugmakers-128328',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.30-128081',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.67-129663',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.88-130496',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.97-127626',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.55-134222',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-seeks-edge-over-rivals-in-severe-asthma-treatment-125011',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-to-pay-$5.52-million-to-resolve-u.s.-sec-foreign-bribery-case-123987',
 "https://uk.investing.com//news/stock-market-news/gsk's-triple-drug-cuts-flare-ups-in-chronic-lung-disease-125322",
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.25-125136',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.44-125577',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.47-123334',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.39-124710',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.06-126839',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.81-127286',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-higher-at-close-of-trade;-investing.com-united-kingdom-100-up-0.30-123340',
 "https://uk.investing.com//news/stock-market-news/astrazeneca-bets-another-$140-million-on-moderna's-'messenger'-drugs-120346",
 "https://uk.investing.com//news/stock-market-news/astrazeneca-hits-2014-pfizer-bid-price,-helped-by-bristol's-woes-119450",
 'https://uk.investing.com//news/stock-market-news/astrazeneca-to-sell-antibiotics-business-to-pfizer-122796',
 'https://uk.investing.com//news/stock-market-news/ftse-retreats-as-pharma-stocks-fall-123027',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.78-120605',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.79-121926',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.08-122719',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.85-120108',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.15-122525',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.19-120834',

 'https://uk.investing.com//news/stock-market-news/astrazeneca-earnings-hit-by-waning-cholesterol-drug-sales-117753',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-resolves-faslodex-patent-litigation-in-u.s.-115005',
 'https://uk.investing.com//news/stock-market-news/ftse-edges-down-from-one-year-high,-earnings-weigh-117785',
 'https://uk.investing.com//news/stock-market-news/ftse-hits-11-month-peak,-miners-lead-114780',
 "https://uk.investing.com//news/stock-market-news/judge-refuses-to-block-generic-versions-of-astrazeneca's-crestor-116187",
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.31-116840',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.33-115890',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.39-117873',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-0.48-117880',
 'https://uk.investing.com//news/politics-news/drug-industry-ceos-lead-new-task-force-on-brexits-pharma-fallout-114055',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.33-114892',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.45-111378',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.60-114262',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.80-114671',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-1.70-113614',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-8.42-112015',
 'https://uk.investing.com//news/stock-market-news/u.k.-shares-lower-at-close-of-trade;-investing.com-united-kingdom-100-down-2.54-112020',
 "https://uk.investing.com//news/stock-market-news/u.s.-spurns-astrazeneca's-nasal-spray-flu-vaccine-as-ineffective-111416",
 'https://uk.investing.com//news/stock-market-news/uk-stocks-reel-in-post-brexit-selloff,-ftse-set-for-worst-day-since-lehman-111503',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-potassium-drug-delayed-by-manufacturing-snag-106318',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-sells-eu,-latam-gout-drug-rights-to-gruenenthal-107362',
 "https://uk.investing.com//news/stock-market-news/astrazeneca-sells-rights-to-anaesthetics-to-south-africa's-aspen-108593",
 'https://uk.investing.com//news/stock-market-news/china-regulator-to-launch-drug-pricing-probe-in-june-106292',
 'https://uk.investing.com//news/stock-market-news/french-drugmaker-sanofi-to-step-up-push-into-china-105834',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.18-107476',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.24-106763',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.66-106204',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.61-108265',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-1.95-110014',
 "https://uk.investing.com//news/stock-market-news/analysis---pfizer's-anacor-deal-showcases-new-wave-of-eczema-therapies-104101",
 "https://uk.investing.com//news/stock-market-news/astrazeneca's-lynparza-fails-in-gastric-cancer-combination-test-104357",
 'https://uk.investing.com//news/stock-market-news/astrazeneca-boosted-by-good-results-for-biotech-asthma-drug-104118',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-drug-wins-orphan-status-in-thyroid-cancer-103287',
 "https://uk.investing.com//news/stock-market-news/insight---pascal-soriot's-big-experiment:-reinventing-astrazeneca-104356",
 'https://uk.investing.com//news/stock-market-news/novartis-splits-drugs-business-into-two,-pharma-chief-to-leave-104255',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-0.98-104015',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-higher-at-close-of-trade;-omx-stockholm-30-up-1.45-102664',
 'https://uk.investing.com//news/stock-market-news/sweden-shares-lower-at-close-of-trade;-omx-stockholm-30-down-0.33-104253',
 'https://uk.investing.com//news/stock-market-news/astrazeneca-a-takeover-target-again-ceo-not-commenting-117792',
]

In [ ]:
# AstraZeneca stock ticker
ticker = 'AZN.L'
# Create a DataFrame to populate while iterating
article_sentiments = pd.DataFrame({'ticker':[],
                                'publish_date':[],
                                'title': [],
                                'body_text': [],
                                'url':[],
                                'neg':[],
                                'neu':[],
                                'pos':[],
                                'compound':[]})
# Loop over all the articles
for link in all_company_urls:
      article = Article(link)
      article.download()

      try:
          article.parse()
          text = article.text

      except:
          print("I didn't get this")
          continue

      # Initialise sentiment analyser
      sid = SentimentIntensityAnalyzer()
      # Get positive, negative, neutral and compound scores
      polarity = sid.polarity_scores(text)

      tmpdic = {'ticker': ticker, 'publish_date': find_date(link), 'title': article.title, 'body_text': article.text, 'url': link}
      # Update ticker with the new entry polarity
      tmpdic.update(polarity)
      # tmpdic now has all keys and values needed to populate the DataFrame
      article_sentiments= article_sentiments.append(pd.DataFrame(tmpdic, index=[0]))
      article_sentiments.reset_index(drop=True, inplace=True)

I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this


In [ ]:
# Show DataFrame of article sentiments data

article_sentiments

,ticker,publish_date,title,body_text,url,neg,neu,pos,compound
0,AZN.L,2021-01-04,UK rolls out Oxford-AstraZeneca vaccine as it ...,LONDON — The U.K. has started rolling out the ...,https://invst.ly/tbyb8,0.018,0.928,0.054,0.9247
1,AZN.L,2021-01-05,The UK is delaying second vaccine shots and it...,In this article PFE Follow your favorite stock...,https://invst.ly/tcjd5,0.045,0.902,0.052,0.7932
2,AZN.L,2021-01-04,UK Stocks-Factors to watch on Jan. 4 By Reuters,"Published Jan 04, 2021 06:31 Updated Jan 04, 2...",https://uk.investing.com//news/stock-market-ne...,0.071,0.875,0.054,-0.8922
3,AZN.L,2020-12-31,NTGR: 3 Small-Cap Stocks for Faster Economic G...,The market finished strong yesterday due to mo...,https://invst.ly/tavzk,0.021,0.835,0.144,0.9992
4,AZN.L,2020-12-31,UK Stocks-Factors to watch on Dec 31 By Reuters,"Published Dec 31, 2020 08:42 Updated Dec 31, 2...",https://uk.investing.com//news/stock-market-ne...,0.050,0.843,0.106,0.9625
...,...,...,...,...,...,...,...,...,...
813,AZN.L,2016-05-17,"Novartis splits drugs business into two, pharm...","Published May 17, 2016 20:38\n\n© Reuters. Fil...",https://uk.investing.com//news/stock-market-ne...,0.064,0.863,0.073,-0.7582
814,AZN.L,2016-05-16,Sweden shares higher at close of trade; OMX St...,"Published May 16, 2016 17:05\n\nSweden shares ...",https://uk.investing.com//news/stock-market-ne...,0.036,0.932,0.031,-0.3818
815,AZN.L,2016-05-09,Sweden shares higher at close of trade; OMX St...,"Published May 09, 2016 17:05\n\n© Reuters. Swe...",https://uk.investing.com//news/stock-market-ne...,0.036,0.933,0.031,-0.3818
816,AZN.L,2016-05-17,Sweden shares lower at close of trade; OMX Sto...,"Published May 17, 2016 17:05\n\n© Reuters. Swe...",https://uk.investing.com//news/stock-market-ne...,0.074,0.912,0.014,-0.9451


In [ ]:
# Save DataFrame

article_sentiments.to_pickle("azn_article_sentiments.pkl")

In [ ]:
article_sentiments.to_csv("azn_article_sentiments.csv", sep=',', encoding='utf-8', header=True)

In [ ]:
# Save URLS to text file

with open('azn_urls.txt', 'w') as f:
    for link in all_company_urls:
        f.write("%s\n" % link)